---
# SVM - Caso Titanic

En este notebook, utilizaremos SVM para entrenar un algoritmo de clasificación que permita establecer un modelo que prediga los sobrevivientes del dataset Titanic

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Carga de Datos y Análisis Exploratorio

In [7]:
df = pd.read_csv('titanic.csv')

In [9]:
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


### Valores perdidos

La cabina no aporta mucho en el análisis, pero la edad es una variable importante para predecir si una persona sobrevivió o no a la trajedia. A continuación, una forma de "visualizar" los valores perdidos.

Haremos una imputación de la edad del pasajero en función del promedio de edad de la clase en la que viajaba.

In [18]:
edades = round(df.groupby('Pclass')['Age'].mean())
edades

Pclass
1    38.0
2    30.0
3    25.0
Name: Age, dtype: float64

In [20]:
# creamos una funcion que imputa la edad en caso de estar nula
def imputar_edad(cols):
    edad = cols[0]
    pclass = cols[1]
    if(pd.isnull(edad)):
        if(pclass==1):
            return 38
        if(pclass==2):
            return 30
        if(pclass==3):
            return 25
    else:
        return edad

In [22]:
df['Age'] = df[['Age','Pclass']].apply(imputar_edad, axis=1)

/var/folders/z7/s8_bm31j1vqbws7fljh6p4xw0000gn/T/ipykernel_22611/3628055789.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  edad = cols[0]
/var/folders/z7/s8_bm31j1vqbws7fljh6p4xw0000gn/T/ipykernel_22611/3628055789.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pclass = cols[1]


In [24]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### Formulación del modelo

En esta sección seleccionaremos las variables que incorporaremos en la matriz de predictores "X" y en el vector de resultados "y", ya que no todas las columnas son de utilidad.

In [ ]:
# Seleccionamos variables predictoras y variable objetivo
df_model = df.copy()
df_model['Embarked'].fillna(df_model['Embarked'].mode()[0], inplace=True)

feature_cols = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = df_model[feature_cols]
y = df_model['Survived']

# Convertimos variables categóricas a dummies
X = pd.get_dummies(X, drop_first=True)

X.head()

### Validación Cruzada

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# División en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=0,
                                                    stratify=y)

### Modelamiento con SVM kernel lineal

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Escalamos los datos (muy importante para SVM)
scaler_linear = StandardScaler()
X_train_scaled_lin = scaler_linear.fit_transform(X_train)
X_test_scaled_lin = scaler_linear.transform(X_test)

# Modelo SVM con kernel lineal
svm_linear = SVC(kernel='linear', C=1.0, random_state=0)
svm_linear.fit(X_train_scaled_lin, y_train)

y_pred_linear = svm_linear.predict(X_test_scaled_lin)

In [ ]:
print("Resultados SVM kernel lineal")
print("Accuracy:", accuracy_score(y_test, y_pred_linear))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred_linear))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred_linear))

### Modelamiento con SVM kernel rgf

In [ ]:
# Modelo SVM con kernel RBF (radial)
scaler_rbf = StandardScaler()
X_train_scaled_rbf = scaler_rbf.fit_transform(X_train)
X_test_scaled_rbf = scaler_rbf.transform(X_test)

svm_rbf = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=0)
svm_rbf.fit(X_train_scaled_rbf, y_train)

y_pred_rbf = svm_rbf.predict(X_test_scaled_rbf)

In [ ]:
print("Resultados SVM kernel RBF")
print("Accuracy:", accuracy_score(y_test, y_pred_rbf))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred_rbf))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred_rbf))

### Optimice los parámetros Gamma, C (regularización), y Kernel

Para esto, utilice una grilla de búsqueda exhaustiva (GridSearchCV). Tome como referencia la siguiente documentación:

https://scikit-learn.org/stable/modules/grid_search.html

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Pipeline: escalado + SVM
pipe_svm = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC())
])

# Definimos grilla de búsqueda
param_grid = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__gamma': [0.01, 0.1, 1, 'scale'],
    'svc__kernel': ['linear', 'rbf', 'poly']
}

grid_search = GridSearchCV(estimator=pipe_svm,
                           param_grid=param_grid,
                           cv=5,
                           scoring='accuracy',
                           n_jobs=-1)

In [ ]:
# Entrenamos la búsqueda en grilla
grid_search.fit(X_train, y_train)

print("Mejores hiperparámetros:")
print(grid_search.best_params_)
print("Mejor accuracy en validación cruzada:", grid_search.best_score_)

# Evaluamos el mejor modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)

print("\nResultados mejor modelo (GridSearchCV)")
print("Accuracy:", accuracy_score(y_test, y_pred_best))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred_best))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred_best))